In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/housesalesprediction/kc_house_data.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['date'], axis=1,inplace=True)

In [ ]:
df['bedrooms'].value_counts()

In [ ]:
df['bedrooms'] = df['bedrooms'].astype(int)

In [ ]:
df['condition']

In [ ]:
Y = df['price']
X = df.drop(['price'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state=0)
print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

In [ ]:
y_train

In [ ]:
df.info()

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# lr.fit(x_train[['bathrooms','bedrooms', 'sqft_living','sqft_lot','waterfront','view','condition','grade','sqft_above', 'sqft_basement','floors','lat','long']], y_train)
lr.fit(x_train[x_train.columns], y_train)
lr.predict(x_test[x_train.columns])

In [ ]:
from sklearn.model_selection import cross_val_score
Rcross = cross_val_score(lr, df[x_train.columns], Y, cv=4)
Rcross
print("The mean of the folds are", Rcross.mean(), "and the standard deviation is" , Rcross.std())

In [ ]:
(-1 * cross_val_score(lr,df[['bathrooms','floors','lat','long']], Y,cv=4,scoring='neg_mean_squared_error'))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()

In [ ]:
yhat_train = lr.predict(x_train[x_train.columns])

In [ ]:
Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_train,yhat_train,'True Value', 'Expected Value', Title)

In [ ]:
yhat_test = lr.predict(x_test)

In [ ]:
Title = 'Distribution  Plot of  Predicted Value Using Testing Data vs Testing Data Distribution'
DistributionPlot(y_test,yhat_test,'True Value', 'Expected Value', Title)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.45, random_state=0)
pr = PolynomialFeatures(degree=5)
x_train_pr = pr.fit_transform(x_train[x_train.columns])
x_test_pr = pr.fit_transform(x_test[x_train.columns])
pr

In [ ]:
poly = LinearRegression()
poly.fit(x_train_pr, y_train)

In [ ]:
yhat_pr = poly.predict(x_test_pr)
yhat_pr[0:5]

In [ ]:
print("Predicted values:", yhat_pr[0:4])
print("True values:", y_test[0:4].values)

In [ ]:
PollyPlot(x_train[x_train.columns], x_test[x_test.columns], y_train, y_test, poly,pr)

In [ ]:
def PollyPlot(xtrain, xtest, y_train, y_test, lr,poly_transform):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))
    
    
    #training data 
    #testing data 
    # lr:  linear regression object 
    #poly_transform:  polynomial transformation object 
 
    xmax=max([xtrain.values.max(), xtest.values.max()])

    xmin=min([xtrain.values.min(), xtest.values.min()])

    x=np.arange(xmin, xmax, 0.1)


    plt.plot(xtrain, y_train, 'ro', label='Training Data')
    plt.plot(xtest, y_test, 'go', label='Test Data')
    plt.plot(x, lr.predict(poly_transform.fit_transform(x.reshape(-1, 1))), label='Predicted Function')
    plt.ylim([-10000, 60000])
    plt.ylabel('Price')
    plt.legend()